# **Aula 01 - Problema de Transportes**

##### **Objetivos da aula**

Introdução ao Gurobi

*   Conceitos básicos do *solver*
*   Como modelar um problema no Gurobi
*   Como resolver e ler a solução do problema

Isso será feito enquanto resolvemos o problema de transportes, um problema simples de variáveis lineares.

###### **O problema**

Problema retidado do livro Pesquisa Operacional (Arenales et al., 2007).

O problema consiste em definir a quantidade de um produto transportada de 2 centros de produção para 3 mercados consumidores. 

O centro de produção 1 possui uma capacidade de 800 unidades, enquanto o centro 2 possui uma capacidade de 1000. Estas capacidades não podem ser excedidas.

A demandas dos mercados 1, 2 e 3 são 500, 400 e 900, respectivamente.

Existe um custo unitário de transporte de cada centro para cada mercado.

O objetivo do problema é encontrar a solução de menor custo que atenda toda a demanda

###### **O modelo**

Variável
*   $x_{ij}$: Quantidade transportada do centro $i$ para o mercado $j$

Função objetiva

$\min \quad 4x_{11}+2x_{12}+5x_{13}+11x_{21}+7x_{22}+4x_{23}$

Restrições de capacidade dos centros

$x_{11} + x_{12} + x_{13} \leq 800$

$x_{21} + x_{22} + x_{23} \leq 1000$

Restrições de atendimento de demanda

$x_{11} + x_{21} = 500$

$x_{12} + x_{22} = 400$

$x_{13} + x_{23} = 900$

Restrições de não-negatividade

$x_{11},x_{12},x_{13},x_{21},x_{22},x_{23} \geq 0$


In [1]:
'''
Este código é usado para chamar a biblioteca gurobipy
Esta é uma versão de testes do Gurobi, e pode ser usada para resolver 
problemas de tamanho pequeno
Caso a sua universidade tenha convênio com o Gurobi, você pode usar
a versão acadêmica
https://www.gurobi.com/
'''
%pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB

Looking in indexes: https://pypi.gurobi.com, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 22.2 MB/s 


In [2]:
'''
O modelo é criado como um objeto usando a função gp.model()
Aqui o modelo se chamará m
'''
m = gp.Model()

'''
As variáveis são criadas dentro do modelo usando a função addVar()

O parâmetro vtype define o tipo de variável, que normalmente são:
CONTINUOUS (contínua)
INTEGER (inteira)
BINARY (binária)

O parâmetro lb define o limitante inferior (lower bound) da variável
Como nossas variáveis são sempre não-negativas, este valor é zero
'''
x_11 = m.addVar(vtype=GRB.CONTINUOUS, lb=0)
x_12 = m.addVar(vtype=GRB.CONTINUOUS, lb=0)
x_13 = m.addVar(vtype=GRB.CONTINUOUS, lb=0)
x_21 = m.addVar(vtype=GRB.CONTINUOUS, lb=0)
x_22 = m.addVar(vtype=GRB.CONTINUOUS, lb=0)
x_23 = m.addVar(vtype=GRB.CONTINUOUS, lb=0)

'''
A função objetivo é criada com a função setObjective(), e é escrita usando uma
notação matemática simples, muito parecida com as usadas no Word e no LaTeX

O parâmetro GRB.MINIMIZE define que esta função será de minimização
Para maximização, usamos GRB.MAXIMIZE
'''

m.setObjective(4*x_11 + 2*x_12 + 5*x_13 +
               11*x_21 + 7*x_22 + 4*x_23,
               GRB.MINIMIZE)

'''
As restrições são criadas com a função addConstr()
A notação é a mesma, e cuidado com os sinais! Para igualdade usamos ==
'''
m.addConstr(x_11 + x_12 + x_13 <= 1000) #Restrição de capacidade do centro 1
m.addConstr(x_21 + x_22 + x_23 <= 1000) #Restrição de capacidade do centro 2
m.addConstr(x_11 + x_21 == 500) #Restrição de demanda do mercado 1
m.addConstr(x_12 + x_22 == 400) #Restrição de demanda do mercado 2
m.addConstr(x_13 + x_23 == 900) #Restrição de demanda do mercado 3

'''
Agora é só otimizar com a função optimize()
'''

m.optimize() 


Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xfbd52b1b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve removed 4 rows and 3 columns
Presolve time: 0.02s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4000000e+03   1.250000e+01   0.000000e+00      0s
       1    6.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds (0.00 work units)
Optimal objective  6.900000000e+03


O problema foi resolvido com sucesso!

A solução tem um custo de 6900 unidades.

Mas... como vemos a solução inteira?

Quanto cada centro envia para cada mercado?

Assim que o modelo é resolvido, todos os parâmetros são salvos dentro do objeto do modelo

A documentação do Gurobi é bem detalhada e você consegue tirar todas as informações necessárias!

Ao longo do curso iremos ver as principais.

In [3]:
'''
O valor da função objetivo é o atributo objVal, chamado diretamente do modelo
Já o valor das variáveis são chamados diretamente delas, através do atributo .X
'''

print("Custo mínimo: "+str(m.objVal))
print("Quantidade enviada:")
print("Centro\tMercado\tQuantidade")
print("1\t1\t"+str(x_11.X))
print("1\t2\t"+str(x_12.X))
print("1\t3\t"+str(x_13.X))
print("2\t1\t"+str(x_21.X))
print("2\t2\t"+str(x_22.X))
print("2\t3\t"+str(x_23.X))

Custo mínimo: 6900.0
Quantidade enviada:
Centro	Mercado	Quantidade
1	1	500.0
1	2	300.0
1	3	0.0
2	1	0.0
2	2	100.0
2	3	900.0


Escrever o modelo restrição por restrição é inviável.

Agora iremos ver como escrever de uma forma muito mais simples!

###### **O modelo generalizado**

Parâmetros
*   $M$: Número de centros de produção
*   $N$: Número de mercados consumidores
*   $a_i$: Capacidade do centro $i$
*   $b_j$: Demanda do mercado $j$
*   $c_{ij}$: Custo de transporte de uma unidade do centro $i$ para o mercado $j$

Variável
*   $x_{ij}$: Quantidade transportada do centro $i$ para o mercado $j$

Função objetiva

$\min \sum_{i=1}^M \sum_{j=1}^N c_{ij}x_{ij}$

Restrições de capacidade dos centros

$\sum_{j=1}^N x_{ij} \leq a_i \quad i = 1,\dots,M$

Restrições de atendimento de demanda

$\sum_{i=1}^M x_{ij} = b_j \quad j = 1,\dots,N$

Restrições de não-negatividade

$x_{ij} \geq 0 \quad i = 1,\dots,M \quad j = 1,\dots,N$

In [4]:
'''
Primeiro definimos os parâmetros do modelo
Para isso, vamos usar vetores e matrizes 
'''

M = 2 
N = 3

a = [800,1000]
b = [500,400,900]
c = [[4,2,5],
    [11,7,4]]

'''
O modelo é criado normalmente
'''

m = gp.Model()

'''
Para criar múltiplas variáveis, usamos a função addVars, que tem como parâmetros
de entrada as dimensões da matriz de variáveis

Como x_ij varia em dois índices, temos umas matriz 2D, portanto, 2 parâmetros
na função addVars

A ordem dos parâmetros é de extrema importância!
'''

x = m.addVars(M,N, vtype=GRB.CONTINUOUS, lb=0)

'''
A função objetivo é escrita normalmente, porém agora escrevemos o somatório com
as matrizes que definimos
'''

m.setObjective(sum(c[i][j]*x[i,j] for i in range(M) for j in range(N)), 
               GRB.MINIMIZE)

'''
Para as restrições, a função addConstrs cria várias restrições do mesmo tipo
Note que em cada uma há um loop no final que diz quantas restrições são criadas
'''

# M restrições de capacidade dos centros
m.addConstrs(sum(x[i,j] for j in range(N)) <= a[i] for i in range(M))
# N restrições de de demanda dos mercados
m.addConstrs(sum(x[i,j] for i in range(M)) == b[j]  for j in range(N))

# Agora é só resolver!
m.optimize() 


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xfbd52b1b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve removed 4 rows and 3 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4000000e+03   1.250000e+01   0.000000e+00      0s
       1    6.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.900000000e+03


# **Exercícios**



1.   O que acontece quando o Centro 1 tem a sua capacidade aumentada para 1000?
2.   O que acontece quando o Mercado 2 aumenta sua demanda para 500?
3.   É correto utilizarmos igualdade nas restrições de capacidade, ao invés da desigualdade $<=$? (Dica: teste o modelo com diferentes valores)